In [1]:
import pandas as pd
import numpy as np

import os

# Load metadata

In [2]:
nl = "\n"

In [3]:
metadata = pd.read_csv(
    os.path.join(
        os.path.abspath(""),
        "gutenberg",
        "SPGC_metadata_20180718.csv"
    ), 
    index_col="id"
)

In [4]:
metadata = metadata[
    (metadata["language"] == "['en']")
][["title", "author"]].dropna()

In [9]:
# Due to memory limitations, we only use the first 10000 books. 
length = 1000
metadata = metadata.iloc[:length]

In [10]:
metadata

,title,author
id,,
PG1,The Declaration of Independence of the United ...,"Jefferson, Thomas"
PG2,The United States Bill of Rights: The Ten Orig...,United States
PG3,John F. Kennedy's Inaugural Address,"Kennedy, John F. (John Fitzgerald)"
PG4,Lincoln's Gettysburg Address: Given November 1...,"Lincoln, Abraham"
PG5,The United States Constitution,United States
...,...,...
PG1068,Personal Memoirs of U. S. Grant — Volume 2,"Grant, Ulysses S. (Ulysses Simpson)"
PG1069,Four Short Stories By Emile Zola,"Zola, Émile"
PG1074,The Sea-Wolf,"London, Jack"


# Load text

In [11]:
# Get all files in the directory and subdirectories that end with .txt
files = [
    os.path.join(root, name)
    for root, dirs, files in os.walk(
        os.path.join(
            os.path.abspath(""),
            "gutenberg",
            "SPGC_tokens_20180718"
        )
    )
    for name in files
    if name.endswith(".txt")
]

k = 5
print(
    f"Found {len(files)} files. {nl}"
    # f"First {k} file(s): {nl}{files[:k]}"
)

Found 55905 files. 


In [12]:
texts = pd.DataFrame(
    [
        [os.path.basename(file).split(".")[0], open(file, "r", encoding="utf8").read().split("\n")]
        for file in files
        if os.path.basename(file).split(".")[0].replace("_tokens", "") in metadata.index
    ],
    columns=["id", "text"]
)

texts["id"] = texts["id"].apply(lambda x: x.replace("_tokens", ""))
texts = texts.set_index("id")

# Add author to text. 
texts = texts.join(metadata["author"], how="inner")

In [13]:
# texts

In [14]:
# Save the texts to a csv file.
texts.to_csv(
    os.path.join(
        os.path.abspath(""),
        f"SPGC_texts_full_{length}_20180718.csv"
    )
)